In [ ]:
import numpy as np
from light_curve.light_curve_py import RainbowFit
import fink_utils.photometry.conversion as convert
import requests
import io
import pandas as pd
import matplotlib.pyplot as plt

def bb_nu(wave_aa, T):
    """Black-body spectral model"""
    nu = 3e10 / (wave_aa * 1e-8)
    return 2 * 6.626e-27 * nu**3 / 3e10**2 / np.expm1(6.626e-27 * nu / (1.38e-16 * T))

In [ ]:
obj = pd.read_csv("ZTF_transfer/ZTF_req_167814.csv")
obj = obj[(obj['FLT'] == 'r') | (obj['FLT'] == 'g')]

#Keep rising part
#obj = obj[(obj['MJD'] < obj['MJD'][np.argmax(obj['FLUXCAL'])])]

flux = obj['FLUXCAL']
fluxerr = obj['FLUXCALERR']
band = obj['FLT']
norm = flux[band == 'r'].max()
flux, fluxerr = flux/norm, fluxerr/norm
mjd = obj['MJD']

In [ ]:
%%time 

band_wave_aa = {"g": 4770.0, "r": 6231.0, "i": 7625.0} #
feature = RainbowFit.from_angstrom(band_wave_aa, with_baseline=False,\
                                   temperature='sigmoid', bolometric='doublexp') 
result = feature(mjd, flux, sigma=fluxerr, band=band)

X = np.linspace(mjd.min()-10, mjd.max()+10, 100)

plt.figure(figsize=(12, 8))

colors = ['green', 'red', 'black']
for idx,i in enumerate(band_wave_aa):
    mask = band == i
    f = flux[mask]
    ferr = fluxerr[mask]
    t = mjd[mask]
    rainbow = feature.model(X, i, *result)
    plt.errorbar(t, f, yerr=ferr, fmt='o', alpha=.7, color=colors[idx])
    plt.plot(X, rainbow, linewidth=5, label=i, color=colors[idx])
    
plt.legend()
print(result)

In [ ]:
feature.

In [1]:
import rainbow_extractor as rx
import pandas as pd

files = ['ZTF21abbzjeq.csv', 'ZTF24aaejsaf.csv', 'ZTF19abxxogb.csv']

lcs = [rx.ZTF_lightcurve("ZTF_transfer/"+file) for file in files]
fex = rx.FeatureExtractor(pd.DataFrame(lcs))
fex.format_all()
fex.full_feature_extraction()

In [7]:
fex.features.keys()

Index(['lc', 'type', 'bolometric', 'temperature', 'fit_error', 'rise_time_exp',
       'pseudo_amplitude_exp', 'reference_time_sigmoid', 'amplitude_sigmoid',
       'rise_time_sigmoid', 'reference_time_linexp', 'amplitude_linexp',
       'rise_time_linexp', 'reference_time_bazin', 'amplitude_bazin',
       'rise_time_bazin', 'fall_time_bazin', 'reference_time_doublexp',
       'amplitude_doublexp', 'time1_doublexp', 'time2_doublexp', 'p_doublexp',
       'T_constant', 'reference_time_sigmoid', 'Tmin_sigmoid', 'Tmax_sigmoid',
       't_color_sigmoid', 'ra', 'dec'],
      dtype='object')

In [ ]:
rx.FeatureExtractor.plot_fit(fex.features.iloc[0], ylim=[-.1, 1.2], xlim=None)